Simple script to choose optimal cards for a cribbage hand. Ignores the play section of the game and only tries to maximize points in the player's hand/crib while minimizing points in the opponent's crib. The script uses a simple montecarlo approach to evaluate the number of points from a given selection of cards. The tricky part is estimating what the opponent will put in the crib-- the simplest implementation of picking two cards randomly works well enough, but slightly improved performance can come (at a large runtime cost) from simulating the opponent picking their own cards to determine what they are likely to put in the crib.

In [1]:
import numpy as np
from itertools import combinations as comb
import time
import random

In [2]:
# Card is a tuple (A = 1, 2-10 = 2-10, J-K = 11-13, Spades=0, Hearts=1, Diamonds=2, Clubs=3)
def print_cards(cards):
    vals = ['A', 2, 3, 4, 5, 6, 7, 8, 9, 10 , 'J', 'Q', 'K']
    suits = ['Spades', 'Hearts', 'Diamonds', 'Clubs']
    out_str = ""
    for val, suit in cards:
        out_str += str(vals[val - 1]) + " " + suits[suit] + "   "
    return out_str.strip()
    
class CardDeck():
    def __init__(self):
        self.cards = []
        for suit in range(4):
            for val in range(1,14):
                self.cards.append((val, suit))
    
    def draw(self, num, remove=False, shuffle=False):
        assert num <= len(self.cards)
        sample = sorted(np.random.choice(len(self.cards), num, replace=False))[::-1]
        cards = []
        for idx in sample:
            cards.append(self.cards[idx])
            if remove:
                del self.cards[idx]
        if shuffle:
            random.shuffle(cards)
        return cards

def deal(n=6):
    deck = CardDeck()
    hand = deck.draw(n, remove=True)
    return hand, deck


def evaluate(hand, start=None):
    points = 0
    # 1 for his knob.
    if start is not None:
        start_suit = start[1]
        for val, suit in hand:
            if suit==start_suit and val==11:
                points += 1 
                break
                
    if start is not None:
        cards = sorted(hand + [start], key=lambda x: x[0])
    else:
        cards = sorted(hand, key=lambda x: x[0])
        
    suits = np.zeros(4)
    vals = np.zeros(13)
    for val, suit in cards:
        suits[suit] += 1
        vals[val - 1] += 1

    # flush (don't need to check for more than 1 flush b/c we have 5 cards)
    if max(suits) >= 4:
        points += int(max(suits))
    
    # multiples
    points += 2 * np.sum(vals == 2) + 6 * np.sum(vals == 3) + 12 * np.sum(vals == 4)

    # straights
    slen = 0
    multiplier = 1
    for val in vals:
        if val > 0:
            slen += 1
            multiplier *= val
        else:
            if slen >= 3:
                points += int(slen * multiplier)
            slen = 0
            multiplier = 1

    # fifteen(s)
    numbers = [min(10, x[0]) for x in cards]
    subsets = list(comb(numbers, 2)) + list(comb(numbers, 3)) + list(comb(numbers, 4)) + list(comb(numbers, 5))
    for subset in subsets:
        if sum(subset) == 15 or (sum(subset) == 5 and 1 in subset):
            points += 2

    return points
    
    

In [3]:
# Evaluation runtime
N = 100
deck = CardDeck()
start = time.time()
for _ in range(N):
    cards = deck.draw(7, remove=False)
    evaluate(cards[:6], cards[-1])
end = time.time()
print("Runtime: %05f sec"% ((end - start) / N))


Runtime: 0.000130 sec


In [4]:
def choose_cards(
    hand, 
    deck, 
    mycrib=True, 
    use_start_card=True, 
    use_crib=True,
    simulate_opponent_crib=False,
    max_crib_its = 1000,
    min_crib_its = 20,
    desired_crib_std = 0.1):
    top_exp = -100
    top_hand = None
    
    for hand_4 in list(comb(hand, 4)):
        # Evaluate expected points from hand
        exp_hand = 0
        if use_start_card:
            for start_card in deck.cards:
                exp_hand += evaluate(list(hand_4), start_card)
            exp_hand = exp_hand / float(len(deck.cards))
        else:
            exp_hand = evaluate(list(hand_4))
        if use_crib:
            # Evaluate expected points from crib
            crib_cards = [x for x in hand if x not in hand_4]
            crib_scores = []
                 
            if simulate_opponent_crib:
                # Assume opponent chooses best cards for self ignoring crib strategy and start card
                for i in range(max_crib_its):
                    new_cards = deck.draw(7, shuffle=True)
                    start_card = new_cards[-1]
                    opponent_hand = new_cards[:6]
                    _, opponent_crib, _ = choose_cards(
                        opponent_hand, 
                        deck, # Unused
                        use_start_card=False, 
                        use_crib=False)
                    crib_scores.append(evaluate(crib_cards + opponent_crib, start_card))
                    if i > min_crib_its and np.std(crib_scores) / np.sqrt(i) <= desired_crib_std:
                        break
            else:     
                # Assume cards put in crib by opponent are truly random
                for i in range(max_crib_its):
                    new_cards = deck.draw(3, shuffle=True)
                    crib =  crib_cards + list(new_cards[:2])
                    start_card = new_cards[2]
                    crib_scores.append(evaluate(crib, start_card))
                    if i > min_crib_its and np.std(crib_scores) / np.sqrt(i) <= desired_crib_std:
                        break

            if mycrib:
                exp = exp_hand + np.mean(crib_scores)
            else:
                exp = exp_hand - np.mean(crib_scores)
        else:
            exp = exp_hand

        if exp > top_exp:
            top_exp = exp
            top_hand = hand_4
    
    top_crib = [x for x in hand if x not in top_hand]
    return top_hand, top_crib, top_exp

        
        

In [5]:
hand, deck = deal()

print("Hand Dealt: %s" % print_cards(hand))
print()

mycrib = False
print("Evaluation without opponent crib simulation, opponent's crib: ")
s = time.time()
hand_4, crib, score = choose_cards(hand, deck, mycrib)
e = time.time()
print("Chosen Cards: %s" % print_cards(hand_4))
print("Chosen Crib: %s" % print_cards(crib))
print("Expected Point Differential: %0.2f" % score)
print("Runtime: %0.2f sec" % (e - s))
print()

print("Evaluation with opponent crib simulation, opponent's crib: ")
s = time.time()
hand_4, crib, score = choose_cards(hand, deck, mycrib, simulate_opponent_crib=True)
e = time.time()
print("Chosen Cards: %s" % print_cards(hand_4))
print("Chosen Crib: %s" % print_cards(crib))
print("Expected Point Differential: %0.2f" % score)
print("Runtime: %0.2f sec" % (e - s))
print()

mycrib = True
print("Evaluation without opponent crib simulation, my crib: ")
s = time.time()
hand_4, crib, score = choose_cards(hand, deck, mycrib)
e = time.time()
print("Chosen Cards: %s" % print_cards(hand_4))
print("Chosen Crib: %s" % print_cards(crib))
print("Expected Point Differential: %0.2f" % score)
print("Runtime: %0.2f sec" % (e - s))
print()

print("Evaluation with opponent crib simulation, my crib: ")
s = time.time()
hand_4, crib, score = choose_cards(hand, deck, mycrib, simulate_opponent_crib=True)
e = time.time()
print("Chosen Cards: %s" % print_cards(hand_4))
print("Chosen Crib: %s" % print_cards(crib))
print("Expected Point Differential: %0.2f" % score)
print("Runtime: %0.2f sec" % (e - s))

Hand Dealt: 7 Clubs   6 Clubs   7 Hearts   A Hearts   K Spades   5 Spades

Evaluation without opponent crib simulation, opponent's crib: 
Chosen Cards: 7 Clubs   6 Clubs   7 Hearts   5 Spades
Chosen Crib: A Hearts   K Spades
Expected Point Differential: 7.53
Runtime: 1.95 sec

Evaluation with opponent crib simulation, opponent's crib: 
Chosen Cards: 7 Clubs   6 Clubs   7 Hearts   5 Spades
Chosen Crib: A Hearts   K Spades
Expected Point Differential: 7.53
Runtime: 9.70 sec

Evaluation without opponent crib simulation, my crib: 
Chosen Cards: 7 Clubs   6 Clubs   7 Hearts   5 Spades
Chosen Crib: A Hearts   K Spades
Expected Point Differential: 16.27
Runtime: 1.83 sec

Evaluation with opponent crib simulation, my crib: 
Chosen Cards: 7 Clubs   6 Clubs   7 Hearts   5 Spades
Chosen Crib: A Hearts   K Spades
Expected Point Differential: 15.84
Runtime: 10.25 sec


In [6]:
# Opponent crib simulation rarely changes the choice of cards. 
# It seems to slightly raise the expected point differential when it's the opponent's crib, but has little effect
# when it is the player's crib. This is likely because our crib simulation makes the opponent likely to discard 
# cards that do not generate points with their hand. In the case of things like pairs/runs/flushes these cards will
# be more likely to generate points with cards that are not in the opponent's hand (eg. the player's crib selection).